In [1]:
# import libraries
from sentence_transformers import SentenceTransformer
from collections import Counter
import requests
import json
import os
from tqdm import tqdm
import pandas as pd
import json

# Change to the parent directory of the notebook
os.chdir('..')

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# EMBEDDINGS_MODEL = SentenceTransformer("dunzhang/stella_en_1.5B_v5", trust_remote_code=True).cuda()
# df = pd.read_csv("Data/dataset.csv", index_col=0)
# print("generating embeddings...")
# # Generate embeddings for each row in the 'tweet_text' column
# tqdm.pandas()

# # Generate embeddings with progress tracking
# df['X_train'] = df['tweet_text'].progress_apply(lambda text: EMBEDDINGS_MODEL.encode(text))

In [2]:
import pickle

# Load using pickle
with open("Data/dataset_mit_embeddings.pkl", "rb") as f:
    df = pickle.load(f)


df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20148 entries, 0 to 20147
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   post_id             20148 non-null  object
 1   tweet_text          20148 non-null  object
 2   key_features        20148 non-null  object
 3   target              16230 non-null  object
 4   label               20148 non-null  object
 5   annotator_1_label   20148 non-null  object
 6   annotator_1_target  13021 non-null  object
 7   annotator_2_label   20148 non-null  object
 8   annotator_2_target  13151 non-null  object
 9   annotator_3_label   20148 non-null  object
 10  annotator_3_target  13080 non-null  object
 11  X_train             20148 non-null  object
 12  label_encoded       20148 non-null  int64 
dtypes: int64(1), object(12)
memory usage: 2.2+ MB


In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])

# Convert 'X_train' column to a list of embeddings
X = df['X_train'].tolist()
y = df['label_encoded'].tolist()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [6]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

# Initialize the XGBoost classifier
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',  # Use 'multi:softmax' for multi-class classification
    num_class=len(label_encoder.classes_),  # Number of classes
    eval_metric='mlogloss',  # Multi-class log loss
    use_label_encoder=False  # Avoid warnings about label encoding
)

# Convert data to proper format if needed
# X_train and X_test are lists of lists (embeddings), so they're ready for XGBoost.

# Train the XGBoost model
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

c:\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:16:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.6278

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.69      0.68      0.68      1285
      normal       0.64      0.75      0.69      1610
   offensive       0.51      0.40      0.45      1135

    accuracy                           0.63      4030
   macro avg       0.61      0.61      0.61      4030
weighted avg       0.62      0.63      0.62      4030



In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the Random Forest classifier
rf_model = RandomForestClassifier(
    n_estimators=100,  # Number of trees in the forest
    criterion='gini',  # Splitting criterion ('gini' or 'entropy')
    random_state=42  # For reproducibility
)

# Train the Random Forest model
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Accuracy: 0.5980

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.66      0.64      0.65      1285
      normal       0.57      0.83      0.68      1610
   offensive       0.56      0.22      0.32      1135

    accuracy                           0.60      4030
   macro avg       0.60      0.56      0.55      4030
weighted avg       0.60      0.60      0.57      4030



In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize classifiers
classifiers = {
    "Random Forest": RandomForestClassifier(
        n_estimators=100, criterion='gini', random_state=42
    ),
    "SVM": SVC(
        kernel='rbf', C=1.0, probability=True, random_state=42
    ),  # Kernel can be 'linear', 'poly', 'rbf', etc.
    "Logistic Regression": LogisticRegression(
        multi_class='multinomial', solver='lbfgs', random_state=42, max_iter=500
    ),
    "K-Nearest Neighbors": KNeighborsClassifier(
        n_neighbors=5, metric='minkowski', p=2  # p=2 for Euclidean distance
    )
}

# Train and evaluate each classifier
for name, model in classifiers.items():
    print(f"\n=== {name} Classifier ===")
    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.4f}")

    # Detailed classification report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred,
          target_names=label_encoder.classes_))


=== Random Forest Classifier ===
Accuracy: 0.5980

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.66      0.64      0.65      1285
      normal       0.57      0.83      0.68      1610
   offensive       0.56      0.22      0.32      1135

    accuracy                           0.60      4030
   macro avg       0.60      0.56      0.55      4030
weighted avg       0.60      0.60      0.57      4030


=== SVM Classifier ===
Accuracy: 0.6665

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.72      0.73      0.72      1285
      normal       0.67      0.79      0.73      1610
   offensive       0.57      0.43      0.49      1135

    accuracy                           0.67      4030
   macro avg       0.65      0.65      0.65      4030
weighted avg       0.66      0.67      0.66      4030


=== Logistic Regression Classifier ===


c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.6372

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.68      0.70      0.69      1285
      normal       0.67      0.73      0.70      1610
   offensive       0.51      0.43      0.47      1135

    accuracy                           0.64      4030
   macro avg       0.62      0.62      0.62      4030
weighted avg       0.63      0.64      0.63      4030


=== K-Nearest Neighbors Classifier ===
Accuracy: 0.5280

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.48      0.85      0.61      1285
      normal       0.68      0.45      0.54      1610
   offensive       0.45      0.28      0.34      1135

    accuracy                           0.53      4030
   macro avg       0.54      0.52      0.50      4030
weighted avg       0.55      0.53      0.51      4030



In [10]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
model = YourModel().to(device)  # Move the model to GPU
tensor = tensor.to(device)

In [14]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Define the parameter grid for Random Forest
param_grid = {
    'n_estimators': [50, 100, 150],  # Number of trees
    'max_depth': [None, 10, 20],  # Maximum depth of each tree
    'min_samples_split': [2, 5, 10],  # Minimum samples to split a node
    'criterion': ['gini', 'entropy']  # Splitting criteria
}

# Initialize the classifier
rf_model = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=rf_model,  # The model to tune
    param_grid=param_grid,  # The parameter grid to search
    scoring='accuracy',  # Evaluation metric
    cv=5,  # Number of cross-validation folds
    verbose=2,  # Controls verbosity
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV on the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and the best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Print the best parameters
print("Best Parameters:", best_params)

# Make predictions using the best model
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# Define classifiers and their parameter grids
models_and_params = {
    "Random Forest": {
        "model": RandomForestClassifier(random_state=42),
        "params": {
            "n_estimators": [50, 100, 150],
            "max_depth": [None, 10, 20],
            "min_samples_split": [2, 5, 10],
            "criterion": ["gini", "entropy"]
        }
    },
    "SVM": {
        "model": SVC(probability=True, random_state=42),
        "params": {
            "C": [0.1, 1, 10],
            "kernel": ["linear", "rbf", "poly"],
            "gamma": ["scale", "auto"]
        }
    },
    "Logistic Regression": {
        "model": LogisticRegression(random_state=42, max_iter=500),
        "params": {
            "C": [0.1, 1, 10],
            "solver": ["lbfgs", "saga"],
            "multi_class": ["auto", "multinomial"]
        }
    },
    "K-Nearest Neighbors": {
        "model": KNeighborsClassifier(),
        "params": {
            "n_neighbors": [3, 5, 7],
            "metric": ["euclidean", "manhattan", "minkowski"],
            "p": [1, 2]
        }
    },
    "XGBoost": {
        "model": xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42),
        "params": {
            "n_estimators": [50, 100, 150],
            "max_depth": [3, 5, 10],
            "learning_rate": [0.01, 0.1, 0.2],
            "subsample": [0.8, 1.0]
        }
    }
}

# Iterate through models and perform GridSearchCV
for name, mp in models_and_params.items():
    print(f"\n=== GridSearch for {name} ===")
    model = mp["model"]
    param_grid = mp["params"]

    # Initialize GridSearchCV
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='accuracy',
        cv=5,
        verbose=2,
        n_jobs=-1
    )

    # Fit GridSearchCV on training data
    grid_search.fit(X_train, y_train)

    # Best parameters and model
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    print(f"Best Parameters for {name}: {best_params}")

    # Make predictions and evaluate
    y_pred = best_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy for {name}: {accuracy:.4f}")

    # Detailed classification report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred,
          target_names=label_encoder.classes_))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb

# Define the parameter grid for XGBoost
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

# Initialize the XGBoost classifier with GPU support
xgb_model = xgb.XGBClassifier(
    use_label_encoder=False,
    eval_metric='mlogloss',
    random_state=42,
    tree_method='gpu_hist',  # Use GPU-accelerated histogram algorithm
    predictor='gpu_predictor'  # Use GPU for predictions
)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=2,
    n_jobs=-1
)

# Fit GridSearchCV on the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and the best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Best Parameters:", best_params)

# Make predictions using the best model
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
